In [90]:
import os
import string
import re

import gensim
from gensim.models import word2vec

import nltk
from nltk import corpus
from nltk.tokenize import TweetTokenizer

In [91]:
folder = 'questions'
files = [file for file in os.listdir(folder) if file.endswith('.txt')]

print(len(files))

31279


In [92]:
questions = []

for file in files:
    with open(os.path.join(folder, file)) as f:
        questions.append(f.read())

Let's save raw, unprocessed questions:

In [93]:
with open('raw_questions.txt', 'w') as f:
    f.write('\n'.join(questions))

## Processing

In [94]:
raw_questions = questions[:]

In [95]:
questions = []
for q in raw_questions:
#     print(q)
    questions.append(''.join([ch for ch in q if ch not in string.punctuation]))

In [96]:
raw_questions[0]

'Кто знает сайты, где можно качать бесплатные mp3?'

In [76]:
print(string.punctuation)
punctuation = '"#$%&\'()*+/:,-.?!;<=>@[\]^_`{|}~'

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [106]:
questions = [''.join([ch for ch in q if ch not in punctuation]) for q in raw_questions]

questions = [q.lower() for q in questions]

print('\n'.join(questions[:5]))
print('\n'.join(questions[-5:]))

кто знает сайты где можно качать бесплатные mp3
почему так часто встречаються люди хамы
почему ногти на больших пальцах рук растут медленнее чем на остальных указательном среднем безымянном и мизинце
у кого есть слайдгитара
как удалить свой вопрос
подскажите пожалйста на сколько опасна киста  в яичнике
ктонибудь прыгал с парашютомкакие ощущения
можно ли приворожить любимогоесли он тебя любитно боится это сказатьпотомушто его засмеют друзья
подскажите какиенибудь прикольные места в петере где можно погулять
скажите пожалуйста  кто из звёзд эстрада русской и зарубежной являются вегеторианцами 


------

In [115]:
stopwords = list(set(corpus.stopwords.words('russian')))

print(len(stop_words))
print(' '.join(stop_words[:25]))

151
разве им сам ним ничего и этом всегда можно много через свою мы какой всех на этот без ней надо три когда ты себе от


------

In [117]:
def question_to_words(raw_question, stopwords = stopwords):
    letters_only = re.sub("[^a-zA-Zа-яА-я0-9]", " ", raw_question) 
    words = letters_only.lower().split()                                                  
    meaningful_words = [w for w in words if not w in stopwords]   

    return(" ".join( meaningful_words ))   

In [118]:
questions = [question_to_words(q) for q in raw_questions]

In [125]:
# nltk.data.load('tokenizers/punkt/russian.pickle')

------

## Попробуем `TweetTokenizer()`

In [128]:
def tokenize_questions(raw_question, tokenizer = TweetTokenizer(), stopwords = None):
    words = tokenizer.tokenize(raw_question.lower())
                     
    if stopwords:
        words = [w for w in words if not w in stopwords]   

    return words

In [148]:
tokens = [tokenize_questions(q) for q in raw_questions]

In [149]:
print('sentenses: ', len(tokens))
print('words: ', len([token for sent in tokens for token in sent]))
print('unique words: ', len(set([token for sent in tokens for token in sent])))

sentenses:  31279
words:  354106
unique words:  53893


In [160]:
for t in tokens[:5]: print(str(t), '\n')

['кто', 'знает', 'сайты', ',', 'где', 'можно', 'качать', 'бесплатные', 'mp3', '?'] 

['почему', 'так', 'часто', 'встречаються', 'люди', 'хамы', '?'] 

['почему', 'ногти', 'на', 'больших', 'пальцах', 'рук', 'растут', 'медленнее', ',', 'чем', 'на', 'остальных', '(', 'указательном', ',', 'среднем', ',', 'безымянном', 'и', 'мизинце', ')', '?'] 

['у', 'кого', 'есть', 'слайд-гитара', '?'] 

['как', 'удалить', 'свой', 'вопрос', '?'] 



Окей, выглядит неплохо, но там наверняка много мусора.

* Добавим `reduce_len = True` (вместо 53893 уникальных слов мы получим 53881): 

In [150]:
tokens_reduce_len = [tokenize_questions(q, TweetTokenizer(reduce_len = True)) 
                     for q in raw_questions]

In [151]:
print('sentenses: ', len(tokens_reduce_len))
print('words: ', len([token for sent in tokens_reduce_len for token in sent]))
print('unique words: ', len(set([token for sent in tokens_reduce_len for token in sent])))

sentenses:  31279
words:  354106
unique words:  53881


In [153]:
print(set([token for sent in tokens for token in sent])
      .difference(set([token for sent in tokens_reduce_len for token in sent])))

{'zzzz', '3200000', '0xc0000005', '2024561111', '300000', '500000', '40000р', 'способствуетtttttt', '330000евро', 'pomogiiiiiiiiiiiiite', '6666', '200000', '100000', '40000', '10000тыс', '30000тыс', '1000000', '10000000', '70000', 'soooooooos', 'aaaaaa', '10000', '20000-25000', '20000', 'xxxxx', '150000', '10000к', '0,000011', '150000тдол', '0x0000001a'}


* Добавим `strip_handles = True` к `reduce_len = True` (вместо 53881 уникальных слов мы получим 53878). 

(Потеряются слова `{'@zakladkis', '@kiska', '@instagram'}`. На мой взгляд, особой важности в них не было.)

In [161]:
tokens_reduce_strip = [tokenize_questions(q, TweetTokenizer(reduce_len = True, strip_handles = True)) 
                    for q in raw_questions]

In [162]:
print('sentenses: ', len(tokens_reduce_strip))
print('words: ', len([token for sent in tokens_reduce_strip for token in sent]))
print('unique words: ', len(set([token for sent in tokens_reduce_strip for token in sent])))

sentenses:  31279
words:  354102
unique words:  53878


In [163]:
print(set([token for sent in tokens_reduce_len for token in sent])
      .difference(set([token for sent in tokens_reduce_strip for token in sent])))

{'@zakladkis', '@kiska', '@instagram'}


------

------

Тут марковчейн для развлечения, пока вопросы парсятся.

In [34]:
# import markovify
# import pymarkovchain

# mv = markovify.Text('\n'.join(questions), state_size = 2)
# for _ in range(50): print(mv.make_sentence(), '\n')

# mc = pymarkovchain.MarkovChain()
# mc.generateDatabase('\n'.join(questions))
# for _ in range(50): print(mc.generateString() + '?', '\n')